In [0]:

import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

import tensorflow_datasets as tfds

In [0]:
imdb, info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [0]:
import numpy as np
train_data,test_data = imdb["train"],imdb["test"]

In [0]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

In [0]:
tf.enable_eager_execution()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

In [0]:
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [0]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 2000
embedding_dim = 16
max_length = 120
trunc_type = "post"
oov_tok = "<OOV>"

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)

In [29]:
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
print(word_index)

{'<OOV>': 1, 'the': 2, 'and': 3, 'a': 4, 'of': 5, 'to': 6, 'is': 7, 'br': 8, 'in': 9, 'it': 10, 'i': 11, 'this': 12, 'that': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'movie': 18, 'but': 19, 'film': 20, "'s": 21, 'on': 22, 'you': 23, 'not': 24, 'are': 25, 'his': 26, 'he': 27, 'have': 28, 'be': 29, 'one': 30, 'all': 31, 'at': 32, 'by': 33, 'they': 34, 'an': 35, 'who': 36, 'so': 37, 'from': 38, 'like': 39, 'her': 40, "'t": 41, 'or': 42, 'just': 43, 'there': 44, 'about': 45, 'out': 46, "'": 47, 'has': 48, 'if': 49, 'some': 50, 'what': 51, 'good': 52, 'more': 53, 'very': 54, 'when': 55, 'she': 56, 'up': 57, 'can': 58, 'b': 59, 'time': 60, 'no': 61, 'even': 62, 'my': 63, 'would': 64, 'which': 65, 'story': 66, 'only': 67, 'really': 68, 'see': 69, 'their': 70, 'had': 71, 'were': 72, 'me': 73, 'well': 74, 'we': 75, 'than': 76, 'much': 77, 'been': 78, 'get': 79, 'bad': 80, 'will': 81, 'people': 82, 'do': 83, 'also': 84, 'other': 85, 'into': 86, 'because': 87, 'great': 88, 'first': 89, 'hi

In [30]:
sequences = tokenizer.texts_to_sequences(training_sentences)
print(sequences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)
print(padded.shape)

(25000, 120)


In [32]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
print(testing_sequences)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)
print(testing_padded)

[[ 153   13   93 ...    1   77  119]
 [   9   94  112 ...    5  153   47]
 [   2    1  956 ...  176    1   47]
 ...
 [   1   23    1 ...    4  211  107]
 [  86    2  644 ...    1   57   47]
 [ 568    9    1 ... 1730  460   47]]


In [0]:
import keras
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [35]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           32000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 43,533
Trainable params: 43,533
Non-trainable params: 0
_________________________________________________________________


In [37]:
num_epo = 10
model.fit(padded,training_labels_final,epochs=num_epo,validation_data=(testing_padded,testing_labels_final))

W0704 19:55:31.532103 140211689559936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 5s 213us/sample - loss: 0.5228 - acc: 0.7184 - val_loss: 0.3696 - val_acc: 0.8353
Epoch 2/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.3440 - acc: 0.8499 - val_loss: 0.3745 - val_acc: 0.8310
Epoch 3/10
25000/25000 [==============================] - 4s 163us/sample - loss: 0.2606 - acc: 0.8964 - val_loss: 0.4168 - val_acc: 0.8148
Epoch 4/10
25000/25000 [==============================] - 4s 163us/sample - loss: 0.1636 - acc: 0.9462 - val_loss: 0.5152 - val_acc: 0.7983
Epoch 5/10
25000/25000 [==============================] - 4s 164us/sample - loss: 0.0872 - acc: 0.9778 - val_loss: 0.6188 - val_acc: 0.7935
Epoch 6/10
25000/25000 [==============================] - 4s 163us/sample - loss: 0.0397 - acc: 0.9935 - val_loss: 0.7459 - val_acc: 0.7897
Epoch 7/10
25000/25000 [==============================] - 4s 165us/sample - loss: 0.0162 - acc: 0.9987 - val_l

In [38]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(2000, 16)


In [0]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [43]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1539], [1539], [1], [], [790], [1], [11], [1], [1], [], [790], [1], [11], [579], [], [11], [579], [], [4], [1783], [4], [1], [11], [1], [1277], [], [], [1], [1005], [1], [968], [579], [790], []]
